In [ ]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
import re
import pickle

In [ ]:
mpaa_results = pd.read_pickle('../data/mpaa_results.pkl')

In [ ]:
titles = []
rating_years = []
mpaa_ratings = []
mpaa_certs = []
reasons = []
others = []

for i in range(0, len(mpaa_results)):
    print(i)
    
    result = mpaa_results[i]
    
    title = re.search('TITLE:(.+) ?\(\d', result)[1].strip()
    rating_year = re.search('TITLE:.+\((\d+)\)R', result)[1].strip()
    mpaa_rating = re.search('RATING:(.+)CERTIFICATE', result)[1].strip()
    mpaa_cert = re.search('CERTIFICATE #:(\d*)', result)[1].strip()
    reason = re.search('REASON:\n?(.*)\n{0,2} ?\n?\xa0D', result)[1].strip().replace('.', '')
    other = re.search('OTHER:(.*)\xa0', result)[1].strip()
    
    titles.append(title)
    rating_years.append(rating_year)
    mpaa_ratings.append(mpaa_rating)
    mpaa_certs.append(mpaa_cert)
    reasons.append(reason)
    others.append(other)
    
mpaa_details = pd.DataFrame(
    {'mpaa_title' : titles,
     'rating_year' : rating_years,
     'mpaa_rating' : mpaa_ratings,
     'mpaa_cert_#' : mpaa_certs,
     'mpaa_reason' : reasons,
     'other_info' : others
    }
)

In [ ]:
mpaa_details

In [ ]:
extra_mpaas = pd.read_pickle('../data/extra_mpaa_results.pkl')

In [ ]:
titles = []
rating_years = []
mpaa_ratings = []
mpaa_certs = []
reasons = []
others = []

for i in range(0, len(extra_mpaas)):
    print(i)
    
    result = extra_mpaas[i]
    
    title = re.search('TITLE:(.+)\(\d', result)[1].strip()
    rating_year = re.search('TITLE:.+\((\d+)\)R', result)[1].strip()
    mpaa_rating = re.search('RATING:(.+)CERTIFICATE', result)[1].strip()
    mpaa_cert = re.search('CERTIFICATE #:(\d*)', result)[1].strip()
    reason = re.search('REASON:\n?(.*)\n{0,2} ?\n?\xa0D', result)[1].strip().replace('.', '')
    other = re.search('OTHER:(.*)\xa0', result)[1].strip()
    
    titles.append(title)
    rating_years.append(rating_year)
    mpaa_ratings.append(mpaa_rating)
    mpaa_certs.append(mpaa_cert)
    reasons.append(reason)
    others.append(other)
    
extra_mpaa_details = pd.DataFrame(
    {'mpaa_title' : titles,
     'rating_year' : rating_years,
     'mpaa_rating' : mpaa_ratings,
     'mpaa_cert_#' : mpaa_certs,
     'mpaa_reason' : reasons,
     'other_info' : others
    }
)

In [ ]:
extra_mpaa_details.loc[extra_mpaa_details['mpaa_title'].str.contains('Cutting')]

In [ ]:
g_plus_results = pd.read_pickle('../data/g_plus_results.pkl').reset_index(drop=True)

In [ ]:
titles = []
rating_years = []
mpaa_ratings = []
mpaa_certs = []
reasons = []
others = []

for i in range(0, len(g_plus_results)):
    print(i)
    
    result = g_plus_results[i]
    
    title = re.search('TITLE:(.+)\(\d', result)[1].strip()
    rating_year = re.search('TITLE:.+\((\d+)\)R', result)[1].strip()
    mpaa_rating = re.search('RATING:(.+)CERTIFICATE', result)[1].strip()
    mpaa_cert = re.search('CERTIFICATE #:(\d*)', result)[1].strip()
    reason = re.search('REASON:\n?(.*)\n{0,2} ?\n?\xa0D', result)[1].strip().replace('.', '')
    other = re.search('OTHER:(.*)\xa0', result)[1].strip()
    
    titles.append(title)
    rating_years.append(rating_year)
    mpaa_ratings.append(mpaa_rating)
    mpaa_certs.append(mpaa_cert)
    reasons.append(reason)
    others.append(other)
    
g_plus_details = pd.DataFrame(
    {'mpaa_title' : titles,
     'rating_year' : rating_years,
     'mpaa_rating' : mpaa_ratings,
     'mpaa_cert_#' : mpaa_certs,
     'mpaa_reason' : reasons,
     'other_info' : others
    }
)

In [ ]:
all_mpaa_details = pd.concat([mpaa_details, extra_mpaa_details, g_plus_details])

In [ ]:
all_mpaa_details.loc[all_mpaa_details['mpaa_title'].str.contains("CERTIFICATE"), 'mpaa_title'] = all_mpaa_details.loc[all_mpaa_details['mpaa_title'].str.contains("CERTIFICATE")]['mpaa_title'].str.split("\(").str[0].str.strip()

In [ ]:
all_mpaa_details = all_mpaa_details.drop_duplicates()

In [ ]:
all_mpaa_details.loc[all_mpaa_details['mpaa_title'] == 'My Sassy Girl']

In [ ]:
# not sure why this wasn't pulled out
mpaa_details.loc[mpaa_details['mpaa_title'] == 'My Sassy Girl', 'mpaa_cert_#'] = '43458'

In [ ]:
# These are both old movies being rerated or versions of movies that were changed prior to release
all_mpaa_details = all_mpaa_details.loc[~(all_mpaa_details['mpaa_cert_#'] == '')]

In [ ]:
# There are 12 certificate numbers used twice for different movies, will have to look at this with merge
len(all_mpaa_details.loc[all_mpaa_details.duplicated(subset = ['mpaa_cert_#'], keep = False)].sort_values('mpaa_cert_#'))

In [ ]:
all_mpaa_details.loc[all_mpaa_details['mpaa_title'].str.contains("CERTIFICATE")]

In [ ]:
mpaa_details.loc[mpaa_details['mpaa_cert_#'] == '31523']

In [ ]:
all_mpaa_details.loc[all_mpaa_details['other_info'].str.contains('Re-Rating')]

In [ ]:
all_mpaa_details = all_mpaa_details.drop_duplicates().reset_index(drop=True)

In [ ]:
all_mpaa_details.to_pickle('../data/mpaa_details.pkl')

In [ ]:
all_mpaa_details['mpaa_rating'].value_counts()

In [ ]:
# These are all movies that were rated prior to the rating reasons being required
all_mpaa_details = all_mpaa_details.loc[~((all_mpaa_details['mpaa_reason'] == '')
              & (all_mpaa_details['mpaa_rating'] != 'G'))]

In [ ]:
# These are all older movies that had the rating symbol updated or for reissue, excluding them from the analysis (100)
all_mpaa_details = all_mpaa_details.loc[~((all_mpaa_details['other_info'] == 'Rating Symbol Changed')
                                       |
                                        all_mpaa_details['other_info'].str.contains('Re-Issue'))]